In [1]:
# !pip install pyarrow

In [2]:
!pip install dfply

In [3]:
import pandas as pd
from dfply import mask, X

In [4]:
# Q1. Downloading the data

In [5]:
jan_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
feb_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [6]:
print(jan_df.shape)  # There are 19 columns
jan_df.head()

(3066766, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [7]:
# Q2. Computing duration

In [8]:
# What's the standard deviation of the trips duration in January?
jan_df['trip_duration'] = jan_df.tpep_dropoff_datetime - jan_df.tpep_pickup_datetime # Compute the trip duration as datetime
jan_df['trip_duration'] = jan_df.trip_duration.dt.total_seconds() / 60 # Compute the trip duration in minutes
jan_df.trip_duration.std() # 42.59 minutes

42.594351241920904

In [9]:
# Q3. Dropping outliers

# Next, we need to check the distribution of the duration variable. There are some outliers. 
# Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

In [10]:
# What fraction of the records left after you dropped the outliers?
jan_df_init = jan_df.copy()
jan_df = jan_df >> mask(X.trip_duration <= 60) >> mask(X.trip_duration >= 1)

print(jan_df.shape[0] * 100 / jan_df_init.shape[0]) # 98% of records remained

98.1220282212598


In [11]:
# Q4. One-hot encoding

# Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

# Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
# Fit a dictionary vectorizer
# Get a feature matrix from it

In [12]:
# What's the dimensionality of this matrix (number of columns)?
from sklearn.feature_extraction import DictVectorizer
jan_df[["PULocationID", "DOLocationID"]] = jan_df[["PULocationID", "DOLocationID"]].astype(str)
records_dict = jan_df[["PULocationID", "DOLocationID"]].to_dict(orient='records')
train_dv = DictVectorizer()
jan_df_ohe = train_dv.fit_transform(records_dict)

In [13]:
len(train_dv.feature_names_) # 515 features
feat_cols = train_dv.feature_names_
feat_cols # Check we only have location IDs

['DOLocationID=1',
 'DOLocationID=10',
 'DOLocationID=100',
 'DOLocationID=101',
 'DOLocationID=102',
 'DOLocationID=106',
 'DOLocationID=107',
 'DOLocationID=108',
 'DOLocationID=109',
 'DOLocationID=11',
 'DOLocationID=111',
 'DOLocationID=112',
 'DOLocationID=113',
 'DOLocationID=114',
 'DOLocationID=115',
 'DOLocationID=116',
 'DOLocationID=117',
 'DOLocationID=118',
 'DOLocationID=119',
 'DOLocationID=12',
 'DOLocationID=120',
 'DOLocationID=121',
 'DOLocationID=122',
 'DOLocationID=123',
 'DOLocationID=124',
 'DOLocationID=125',
 'DOLocationID=126',
 'DOLocationID=127',
 'DOLocationID=128',
 'DOLocationID=129',
 'DOLocationID=13',
 'DOLocationID=130',
 'DOLocationID=131',
 'DOLocationID=132',
 'DOLocationID=133',
 'DOLocationID=134',
 'DOLocationID=135',
 'DOLocationID=136',
 'DOLocationID=137',
 'DOLocationID=138',
 'DOLocationID=139',
 'DOLocationID=14',
 'DOLocationID=140',
 'DOLocationID=141',
 'DOLocationID=142',
 'DOLocationID=143',
 'DOLocationID=144',
 'DOLocationID=145',

In [14]:
# Q5. Training a model

# Now let's use the feature matrix from the previous step to train a model.

# Train a plain linear regression model with default parameters
# Calculate the RMSE of the model on the training data

In [15]:
# What's the RMSE on train?
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Fit model
lr = LinearRegression()
lr.fit(jan_df_ohe, jan_df.trip_duration)
y_pred = lr.predict(jan_df_ohe)
print(mean_squared_error(jan_df.trip_duration, y_pred, squared=False)) # I get 7.649 (7.65) and the closest answer is 7.64

7.649261027919939


In [16]:
# Q6. Evaluating the model

# Now let's apply this model to the validation dataset (February 2023).
feb_df['trip_duration'] = feb_df.tpep_dropoff_datetime - feb_df.tpep_pickup_datetime # Compute the trip duration as datetime
feb_df['trip_duration'] = feb_df.trip_duration.dt.total_seconds() / 60 # Compute the trip duration in minutes

# Remove outliers
feb_df = feb_df >> mask(X.trip_duration <= 60) >> mask(X.trip_duration >= 1)

# One-hot encode the val set
feb_df[["PULocationID", "DOLocationID"]] = feb_df[["PULocationID", "DOLocationID"]].astype(str)
records_dict = feb_df[["PULocationID", "DOLocationID"]].to_dict(orient='records')
vaL_dv = DictVectorizer()
feb_df_ohe = vaL_dv.fit_transform(records_dict)
feb_df_ohe = pd.DataFrame.sparse.from_spmatrix(feb_df_ohe, columns=vaL_dv.feature_names_)

In [17]:
# Add features in the train set that are not in the val set
for col in train_dv.feature_names_:
    if col not in feb_df_ohe.columns:
        feb_df_ohe[col] = 0

In [18]:
# Check that the validation set has all necessary cols
feb_df_ohe[train_dv.feature_names_]

,DOLocationID=1,DOLocationID=10,DOLocationID=100,DOLocationID=101,DOLocationID=102,DOLocationID=106,DOLocationID=107,DOLocationID=108,DOLocationID=109,DOLocationID=11,...,PULocationID=9,PULocationID=90,PULocationID=91,PULocationID=92,PULocationID=93,PULocationID=94,PULocationID=95,PULocationID=96,PULocationID=97,PULocationID=98
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2855946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2855947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2855948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2855949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Convert back to sparse matrix to consume less computing power
from scipy.sparse import csr_matrix
val_matrix = csr_matrix(feb_df_ohe[train_dv.feature_names_].astype(pd.SparseDtype("float64",0)).sparse.to_coo())

In [20]:
# What's the RMSE on validation?
y_pred = lr.predict(val_matrix)
print(mean_squared_error(feb_df.trip_duration, y_pred, squared=False)) # I get 7.81

7.811832638273232
